# Problem 3.9

In [4]:
qui import delim ./data/chapter03/Camerer_JPEsubset_1998, clear
set seed 1234567

## Part b

In [5]:
rename treatment D
rename pair block
rename preexperimentbets covs

In [6]:
// calculate probs under block assignment
qui bysort block: egen probs=mean(D)

// permuation to calculate F stat and one-side P value
ritest D e(F), strata(block) reps(10000) right nodots: ///
regress D covs

// p.value
di el(r(p),1,1)




      Source |       SS           df       MS      Number of obs   =        34
-------------+----------------------------------   F(1, 32)        =      0.02
       Model |  .005024372         1  .005024372   Prob > F        =    0.8914
    Residual |  8.49497563        32  .265467988   R-squared       =    0.0006
-------------+----------------------------------   Adj R-squared   =   -0.0306
       Total |         8.5        33  .257575758   Root MSE        =    .51524

------------------------------------------------------------------------------
           D |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
        covs |  -.0000386   .0002809    -0.14   0.891    -.0006109    .0005336
       _cons |   .5137818   .1335793     3.85   0.001     .2416896     .785874
------------------------------------------------------------------------------

      command:  regress D covs
        _pm_1:  

## Part c

In [7]:
rename experimentbets change

In [8]:
tabstat change, by(D) stat(mean) save	
di "ATE ="%180.4f el(r(Stat2),1,1)-el(r(Stat1),1,1)



Summary for variables: change
     by categories of: D 

       D |      mean
---------+----------
       0 |  571.4118
       1 |  461.2353
---------+----------
   Total |  516.3235
--------------------

ATE =                 -110.1765


## Part d

In [9]:
qui bysort block (D): gen pair_diff = change - change[_n+1]
mean(pair_diff)




Mean estimation                   Number of obs   =         17

--------------------------------------------------------------
             |       Mean   Std. Err.     [95% Conf. Interval]
-------------+------------------------------------------------
   pair_diff |   110.1765   104.8377     -112.0695    332.4225
--------------------------------------------------------------


## Part e

In [10]:
cap program drop ate_block	
program define ate_block, rclass
args Y D probs
tempvar ipw
gen `ipw' = .
// calculate inverse probability weight under block assignment
replace `ipw' = `D'/`probs' + (1-`D')/(1-`probs')
qui reg `Y' `D' [iw=`ipw']
return scalar ate=_b[`D']
end 

In [11]:
ritest D r(ate), strata(block) reps(10000) nodots: ///
ate_block change D probs

(34 missing values generated)
(34 real changes made)

      command:  ate_block change D probs
        _pm_1:  r(ate)
  res. var(s):  D
   Resampling:  Permuting D
Clust. var(s):  __00000A
     Clusters:  34
Strata var(s):  block
       Strata:  17

------------------------------------------------------------------------------
T            |     T(obs)       c       n   p=c/n   SE(p) [95% Conf. Interval]
-------------+----------------------------------------------------------------
       _pm_1 |  -110.1765    3170   10000  0.3170  0.0047  .3078845   .3262222
------------------------------------------------------------------------------
Note: Confidence interval is with respect to p=c/n.
Note: c = #{|T| >= |T(obs)|}


In [12]:
// ate
di el(r(b),1,1)

-110.17647


In [13]:
// p.value.twosided
di el(r(p),1,1)

.317
